# Delays for departure airports

In [ ]:
%load_ext sql
from flights import settings
%sql $settings.DB_URL

In [ ]:
%%sql
select departure_airport,
       avg(actual_departure - scheduled_departure) as avg_delay
from flight
group by departure_airport;

## Using Pandas

In [3]:
import pandas as pd
import numpy as np
from flights import models
from flights.database import db_session

In [4]:
SPARSITY = 100
statement = models.all_flights().filter(models.Flight.id % SPARSITY == 0).statement
flights = pd.read_sql(statement, db_session.bind)

In [5]:
flights['delay'] = (flights['actual_departure'] - flights['scheduled_departure']) \
                    .apply(lambda delay: delay.total_seconds())

In [8]:
stats = flights.groupby('departure_airport')['delay'] \
               .aggregate([np.mean, min, max])

In [9]:
stats

,mean,min,max
departure_airport,,,
AAL,603.529412,-840.0,9120.0
AAQ,110.000000,-60.0,300.0
AAR,86.400000,-360.0,1800.0
AAT,-534.782609,-1500.0,1980.0
ABA,300.000000,-420.0,1080.0
ABE,3990.000000,-1080.0,23520.0
ABI,3643.448276,-660.0,20220.0
ABJ,12661.500000,-840.0,40080.0
ABQ,3029.021739,-780.0,30300.0


In [10]:
from flights import redis

In [11]:
for stat in stats.iterrows():
  redis.set('departure_%s' % stat[0], {
            'mean': stat[1]['mean'],
            'max': stat[1]['max'],
            'min': stat[1]['min'],
            })

![I have no idea what I'm doing](http://cathyreisenwitz.com/wp-content/uploads/2016/01/no.jpg)